In [1]:
"""
外部音声の録音・保存とデータセットの作成
実装を極力AE.Pyと共通化することで移植性を確保
"""

#%%
# cording = UTF-8
import os,wave,datetime,configparser
import scipy,pyaudio
import numpy as np
import matplotlib.pyplot as plt

print ("Scipy version:{0}".format(scipy.__version__))
print ("Pyaudio version:{0}".format(pyaudio.__version__))
print ("Numpy version:{0}".format(np.__version__))


###########################グローバル変数の初期値###########################

bitrate = pyaudio.paInt16
sr = 22050  #サンプリングレート。もともとAE.Pyでは22050で固定している
rec_length = 3
a_index = None

base_dir = "../"
save_dir = os.path.join(base_dir,"data/bulk_wav")
disp_spg = False    #録音後にスペクトラムを表示するかどうか Falseだと波形表示


Scipy version:1.5.0
Pyaudio version:0.2.11
Numpy version:1.18.5


In [2]:
#############################定義系オブジェクト#############################

#設定値の読み込み・書き出し処理
class init_val:
    def __init__(self):
        pass

    #設定値のセット
    def def_ini(self):
        x = configparser.ConfigParser()

        x["General"] = {
            "base_dir" : base_dir,
            "save_dir" : save_dir,
            "disp_spg" : disp_spg
        }

        x["Rec_param"] = {
            "bitrate" : bitrate,
            "sr" : sr,
            "rec_length" : rec_length,
            #"a_index" : a_index,
        }

        #オーディオインデックスの処理 CongifparserはNoneを扱えないため空白にする
        if a_index == None:
            x["Rec_param"]["a_index"] = ""
        else:
            x["Rec_param"]["a_index"] = a_index

        return x

    #iniファイルの保存
    def save_ini(self,cfg):
        with open("Rec.ini","w") as cfgfile:
            cfg.write(cfgfile)

    #iniファイルの読み出し
    def load_ini(self):
        x = configparser.ConfigParser()
        x.read("./Rec.ini")

        return x

    #iniインスタンスからの変数読み出し
    def set_ini(self,cfg):
        bitrate = cfg.getint("Rec_param","bitrate")
        sr = cfg.getint("Rec_param","sr")
        rec_length = cfg.getint("Rec_param","rec_length")
        #a_index = cfg.getint("Rec_param","a_index")
        disp_spg = cfg.getboolean("General","disp_spg")

        base_dir = cfg.get("General","base_dir")
        save_dir = cfg.get("General","save_dir")

        #オーディオインデックス番号の処理
        a_index = cfg.get("Rec_param","a_index")
        if a_index == "":
            a_index = None
        else:
            a_index = int(a_index)

        return bitrate,sr,rec_length,a_index,disp_spg,base_dir,save_dir


#初期設定インデックス定義
class Def_index:
    def __init__(self):
        pass

    #オーディオインデックスの取得
    def select_indexes(self):
        p = pyaudio.PyAudio()
        print("***List of available audio devices:***")
        for i in range(p.get_device_count()):
            print(i,p.get_device_info_by_index(i).get("name"),sep = " - ")
        x = int(input("Select Audio device Index No."))
        print("***Audio device #{0} selected***".format(x))
        del p
        return x

#録音関連
class Erem_rec:
    def __init__(self):
        pass
    
    #録音を行う
    def proc_rec(self,bitrate,sr,rec_length,a_index):
        p = pyaudio.PyAudio()

        #ストリームの開始
        stream = p.open(format = bitrate,
                        channels = 1,   #モノラル
                        rate = sr,
                        input = True,
                        input_device_index = a_index,
                        frames_per_buffer = 1024)   #ストリームサイズ1204固定
        
        #フレームサイズごとに音声を録音
        print("Now Recording...")
        x = []
        for i in range(0,int(sr / 1024 * rec_length)):
            x.append(stream.read(1024))
        
        #ストリームを終了
        print("finished!")
        stream.stop_stream()
        stream.close()
        p.terminate()

        #リスト型になっているxをまとめる
        x = b"".join(x)

        #後処理と出力
        del p,stream
        return x

    #波形の描画
    def vis_waveform(self,waveform):
        plt.plot(waveform)
        plt.show()

    #スペクトログラムの描画(デフォルトはオフ)
    def vis_spectrogram(self,waveform,sr):
        x = np.arange(0)
        from scipy import signal    #Scipy1.5.0以後この構文が必要になった
        freq,time,x = scipy.signal.spectrogram(
            waveform,
            fs = sr,
            window = np.hamming(1024),
            nfft = 1024,
            scaling = "spectrum",
            mode = "magnitude"
        )
        del freq,time
        from matplotlib.colors import LogNorm
        plt.pcolormesh(x,norm = LogNorm())
        plt.colorbar()
        plt.yscale("Log")
        plt.show()

    #Wavファイルへの書き出し
    def save_rec(self,bulkwave,save_dir,bitrate,sr):
        #ファイルネームの生成 保存時の時間で生成する
        dt = datetime.datetime.now()
        filename = dt.strftime("%Y%m%d%H%M%S") + ".wav"

        #実際の保存処理 waveオブジェクトをほぼリファレンス通りに使う
        r_path = os.getcwd()    #保存処理終了後のディレクトリ戻り先の取得
        os.chdir(save_dir)

        p = pyaudio.PyAudio()
        x = wave.open(filename,"wb")
        x.setnchannels(1)
        x.setsampwidth(p.get_sample_size(bitrate))
        x.setframerate(sr)
        x.writeframes(bulkwave)
        x.close()

        del p,x
        print("Saved! Filename:{0}".format(filename))

        #カレントディレクトリを元に戻す
        os.chdir(r_path)
        del r_path

#############################処理系オブジェクト#############################

#録音からセーブまでの一貫処理
class Audio_Recoder(Erem_rec):
    #何度も呼び出されるのでコンストラクタで引数を設定しておく
    def __init__(self,bitrate,sr,rec_length,a_index,save_dir):
        self.bitrate = bitrate
        self.sr = sr
        self.rec_length = rec_length
        self.a_index = a_index
        self.save_dir = save_dir

    def exec(self):
        #録音処理
        x = super().proc_rec(
            self.bitrate,self.sr,self.rec_length,self.a_index
        )

       #録音結果の表示
        form_wave = np.frombuffer(
            x,dtype="int16"
            ) / float(
                (np.power(2, 16) / 2) - 1
                )   #データ表示用にNumpy配列に変換
        if disp_spg == False:
            super().vis_waveform(form_wave)
        else:
            super().vis_spectrogram(form_wave,self.sr)
        del form_wave

        #セーブ
        super().save_rec(x,self.save_dir,self.bitrate,self.sr)   


In [3]:
ini = init_val()
cfg = ini.def_ini()
ini.save_ini(cfg)

In [4]:
del cfg,bitrate,sr,rec_length,a_index,disp_spg,base_dir,save_dir

In [5]:
cfg = ini.load_ini()

In [6]:
cfg.sections()

['General', 'Rec_param']

In [7]:
cfg.items()

ItemsView(<configparser.ConfigParser object at 0x000001FBF7017248>)

In [8]:
bitrate,sr,rec_length,a_index,disp_spg,base_dir,save_dir = ini.set_ini(cfg)